In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta

# Function to convert space values to -1
def space_to_int(value):
    if value == ' ':
        return -1
    else:
        return int(value)  

# Function to lookup dates
def create_date_lookup(df, fields):
    field_val_set = set()
    for field in fields:
        field_val_set = field_val_set | set(df[field])
        
    dates_dict = {date:pd.to_datetime(date) for date in field_val_set}
    dates_dict[' '] = np.nan
    return dates_dict

# Path to data file and information to process data file
data_file = 'C:/Users/Lucas/Documents/NDACAN SRI/Data Request/fc_all.csv'
columns_to_use = (0,3,6,7,24,42,65,76,77,78,79,86,91,92,95,96)
date_columns = ['DLstFCDt','LatRemDt','CurSetDt','DoDFCDt']
data_types = {'RecNumbr':str, 'Entered':np.bool, 'Exited':np.bool}
convert_dict = {i:space_to_int for i in ['TOTALREM','CURPLSET','DISREASN','LatRemLOS','AgeAtLatRem','AgeAtEnd'] }

# Load data with reasonable chunksize iterator
chunksize_param = 500000

data_load = pd.read_csv(
    data_file,
    usecols=columns_to_use,
    dtype=data_types,
    converters=convert_dict,
    chunksize=chunksize_param)

data_list = []

for i, data in enumerate(data_load):
    print(str(i*chunksize_param))
    data_list.append(data)

print('Concatenating...')
data_df = pd.concat(data_list)
print('Complete!')

# Convert to datetimes
dates_dict = create_date_lookup(data_df, date_columns)
for var in date_columns:
    print(var)
    data_df[var] = data_df[var].apply(lambda v: dates_dict[v])

# Rename the first column because of byte-order-mark thing at beginning
data_df.rename(columns={'\ufeffDataYear':'DataYear'},inplace=True)

# Test that it all worked
data_df.head()

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
10000000
10500000
11000000
Concatenating...
Complete!
DLstFCDt
LatRemDt
CurSetDt
DoDFCDt


,DataYear,St,FIPSCODE,RecNumbr,CURPLSET,DISREASN,DLstFCDt,LatRemDt,CurSetDt,DoDFCDt,LatRemLOS,AgeAtLatRem,AgeAtEnd,Entered,Exited
0,2000,AL,8,000000001907,5,0,NaT,1983-04-16,1999-08-21,NaT,6385,0,17,False,False
1,2000,AL,8,000000001997,3,4,NaT,1981-06-07,1981-06-07,2000-07-29,6992,1,20,False,True
2,2000,AL,1073,000000002014,5,1,1991-08-08,1999-04-07,1999-04-07,2000-03-09,337,27,27,False,True
3,2000,AL,8,000000002074,6,-1,1984-10-30,1984-12-29,1997-08-01,2000-03-15,5555,5,21,False,True
4,2000,AL,8,000000003214,5,0,NaT,1982-10-11,1991-11-01,NaT,6553,0,18,False,False


In [2]:
print('Calculating P1 altered indicator...')

# Sort data by state, child, and year to create permanency indicator
data_df.sort(['St','RecNumbr','DataYear'], inplace=True)

# Create conditional for permanency denominator
P1_denom_permanency = (data_df['Entered']) & (data_df['AgeAtLatRem'] < 17) & ( (data_df['LatRemLOS'] >= 8) | (~data_df['Exited']) )
first_entry = (data_df['TOTALREM'] == 1)
data_df['Perm_denom'] = first_entry & P1_denom_permanency

# Create field for permanency date
data_df['PermDt'] = np.nan

data_df.loc[(data_df['Exited']) & (data_df['LatRemLOS'] <= 365) & (data_df['DISREASN'].isin([1,2,3,5])) & (data_df['AgeAtEnd'] < 18), 'PermDt'] = data_df['DoDFCdt']


P1_denom_reentry = (data_df['Exited']) & (data_df['LatRemLOS'] <= 365) & (data_df['DISREASN'].isin([1,2,3,5])) & (data_df['AgeAtEnd'] < 17)
P1_num_reentry = (False)



# Create conditional for permanency over the next two years, and including THVA
for year_offset in (-1,-2):

    data_df_lead = data_df.shift(year_offset)
    same_child = (data_df_lead['St'] == data_df['St']) & (data_df_lead['RecNumbr'] == data_df['RecNumbr'])
    permanency = (data_df_lead['Exited']) & (data_df_lead['DISREASN'].isin([1,2,3,5]))
    reentry = (data_df_lead['LatRemDt'] != data_df['LatRemDt'])
    
    age_exit_LT18 = (data_df_lead['AgeAtEnd'] < 18)
    age_exit_LT17 = (data_df_lead['AgeAtEnd'] < 17)
    
    perm_timeframe = (data_df_lead['DoDFCDt'] - data_df['LatRemDt'] <= timedelta(days=365))
    reentry_timeframe = (data_df_lead['LatRemDt'] - data_df_lead['DLstFCDt'] <= timedelta(days=365))
    THVA = (data_df_lead['CURPLSET'] == 8) & (data_df_lead['CurSetDt'] - data_df['LatRemDt'] + timedelta(days=30) <= timedelta(days=365))

    P1_num_permanency = P1_num_permanency | ( same_child & permanency & age_exit_LT18 & (~reentry) & (perm_timeframe | THVA) )
    P1_denom_reentry = P1_denom_reentry | ( same_child & permanency & age_exit_LT17 & (~reentry) & (perm_timeframe | THVA) )
    P1_num_reentry = P1_num_reentry | ( same_child & reentry & reentry_timeframe )
    
# Create fields for denominator/numerator in permanency and reentry -- first entries only

data_df['P1_num_perm'] = P1_denom_permanency & P1_num_permanency
data_df['P1_denom_reentry'] = P1_denom_permanency & P1_denom_reentry
data_df['P1_num_reentry'] = P1_denom_permanency & P1_denom_reentry & P1_num_reentry

print('Complete!')

Calculating P1 indicator...
Complete!


In [3]:
# Year, Year&State, Year&State&FIPS
analyses = [
    ('DataYear', 'perm_first_by_year'),
    (['St','DataYear'], 'perm_first_by_year_state'),
    (['St','FIPSCODE','DataYear'], 'perm_first_by_year_fips')
]

for item, name in analyses:
    # Sum up entries and permanency by parameters, calculate percentage
    P1_df = data_df.groupby(item)['P1_denom_perm','P1_num_perm','P1_denom_reentry','P1_num_reentry'].sum()
    P1_df['P1_perm_percent'] = P1_df['P1_num_perm'] / P1_df['P1_denom_perm']
    P1_df['P1_reentry_percent'] = P1_df['P1_num_reentry'] / P1_df['P1_denom_reentry']

    # Save to csv
    P1_df.to_csv('C:/Users/Lucas/Documents/NDACAN SRI/Analysis/' + name + '.csv')

In [6]:
# Sum up entries and permanency by year and state, calculate percentage -- FY2011 and 2012
P1_df = data_df[data_df['DataYear'].isin([2011,2012])].groupby(['St','DataYear'])['P1_denom_perm','P1_num_perm','P1_denom_reentry','P1_num_reentry'].sum()
P1_df['P1_perm_percent'] = P1_df['P1_num_perm'] / P1_df['P1_denom_perm']
P1_df['P1_reentry_percent'] = P1_df['P1_num_reentry'] / P1_df['P1_denom_reentry']

# Add Children's Bureau calculations of CFSR permanency and reentry performance
cb_cfsr = pd.read_excel('C:/Users/Lucas/Documents/NDACAN SRI/Analysis/CFSR_P1.xlsx')
P1_df_merged = pd.merge(P1_df.unstack(), cb_cfsr, how='left', left_index=True, right_on='St')
P1_df_merged.set_index(['St'], inplace=True)

# Add fields for differences in P1&Reentry in each year
for year in [2011,2012]:
    variable_names = [ start + ', ' + str(year) for start in ['P1_Diff','Re_Diff'] ]
    P1_df_merged[variable_names[0]] = P1_df_merged['P1_perm_percent', year] - P1_df_merged['CB_P1_percent']
    P1_df_merged[variable_names[1]] = P1_df_merged['P1_reentry_percent', year] - P1_df_merged['CB_Re_percent']

# Save to csv
P1_df_merged.to_csv('C:/Users/Lucas/Documents/NDACAN SRI/Analysis/P1_indicator_by_year_state_11-12.csv')

In [7]:
P1_df = data_df[(data_df['St']=='NM') & (data_df['DataYear'] >= 2011)].to_csv('C:/Users/Lucas/Documents/NDACAN SRI/Analysis/NM_detail.csv')